In [20]:
import pandas as pd
import json

In [21]:
df = pd.read_csv('data/data.csv')

In [22]:
df['dx'] = df['dx'].str.strip().str.lower()

In [23]:
df['dx'].value_counts()

dx
early gastric cancer                        366
rectal cancer                               270
hepatocellular carcinoma                    247
colon cancer, sigmoid                       207
thyroid nodule                              202
                                           ... 
systemic lupus erythematosus                  1
anal fissure                                  1
malignant neoplasm of salivary gland          1
biliary acute pancreatitis                    1
colon submucosal tumor, unknown behavior      1
Name: count, Length: 842, dtype: int64

In [24]:
diagnoses = pd.Series(df['dx'].unique())

In [25]:
single_worded_diagnoses = pd.Series([d for d in diagnoses if ' ' not in d])
len(single_worded_diagnoses)

67

In [26]:
mapping = {}
for d in single_worded_diagnoses:
    for x in diagnoses[diagnoses.str.contains(d)]:
        if x != d:
            mapping[x] = d

with open('mapping.json', 'w') as f:
    json.dump(mapping, f)

In [27]:
diagnoses = diagnoses.replace(mapping)
diagnoses = pd.Series(diagnoses.unique())
df['dx'] = df['dx'].replace(mapping)

In [28]:
two_worded_diagnoses = pd.Series([d for d in diagnoses if len(d.split()) == 2])
len(two_worded_diagnoses)

207

In [29]:
mapping = {}
for d in two_worded_diagnoses:
    for x in diagnoses[diagnoses.str.contains(d)]:
        if x != d:
            mapping[x] = d

with open('mapping.json', 'w') as f:
    json.dump(mapping, f)

In [30]:
exemptions = ["colorectal cancer", "gallbladder cancer"]
mapping = {k: v for k, v in mapping.items() if k not in exemptions}
diagnoses = diagnoses.replace(mapping)
df['dx'] = df['dx'].replace(mapping)
diagnoses = pd.Series(diagnoses.unique())

In [31]:
three_worded_diagnoses = pd.Series([d for d in diagnoses if len(d.split()) == 3])
len(three_worded_diagnoses)

107

In [32]:
mapping = {}
for d in three_worded_diagnoses:
    for x in diagnoses[diagnoses.str.contains(d)]:
        if x != d:
            mapping[x] = d

with open('mapping.json', 'w') as f:
    json.dump(mapping, f)

In [33]:
further_aggregated = {
    'gastric cancer': 'gastric cancer',
    'inguinal hernia': 'inguinal hernia',
    'renal cell carcinoma': 'renal cell carcinoma',
}
for k, v in mapping.items():
    for k1, v1 in further_aggregated.items():
        if k1 in k:
            mapping[k] = v1

with open('mapping.json', 'w') as f:
    json.dump(mapping, f)

In [34]:
diagnoses = diagnoses.replace(mapping)
diagnoses = pd.Series(diagnoses.unique())
df['dx'] = df['dx'].replace(mapping)

In [35]:
# df.loc[df['dx'].isin(df['dx'].value_counts()[df['dx'].value_counts() < 2].index), 'dx'] = 'other'
df = df.loc[df['dx'].isin(df['dx'].value_counts()[df['dx'].value_counts() > 9].index)]

In [36]:
df['dx'] = df['dx'].str.capitalize()
df.groupby('dx').size().sort_values(ascending=False)

dx
Colon cancer                              549
Early gastric cancer                      366
Lung cancer                               315
Rectal cancer                             270
Gallbladder stone                         260
                                         ... 
Adenocarcinoma of lung                     11
Common bile duct cancer, distal            10
Crohn's disease                            10
Antenatal care for high risk pregnancy     10
Melanoma                                   10
Length: 90, dtype: int64

In [37]:
df['dx'].value_counts()

dx
Colon cancer                              549
Early gastric cancer                      366
Lung cancer                               315
Rectal cancer                             270
Gallbladder stone                         260
                                         ... 
Appendix mucocele                          11
Common bile duct cancer, distal            10
Antenatal care for high risk pregnancy     10
Crohn's disease                            10
Melanoma                                   10
Name: count, Length: 90, dtype: int64

In [38]:
df.to_csv('data/data_cleaned_filtered.csv', index=False)